In [168]:
import torch.nn as nn
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

In [169]:
# cols = [i for i in range(1, 15)]
# housing_data = pd.read_csv('housing.csv',delimiter=r"\s+", names=cols)
# targets = housing_data[14]
# data = housing_data.drop(labels=14, axis=1)
# #data

bos = load_boston()
bos.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [170]:
df = pd.DataFrame(bos.data)
df.columns = bos.feature_names
df['Price'] = bos.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [171]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [172]:
# ## Standardising the data
# data = df[df.columns[:-1]]
# data = data.apply(
#     lambda x: (x - x.mean()) / x.std()
# )

# data['Price'] = df.Price

In [174]:
data = df.drop('Price', axis=1).to_numpy()
targets = df['Price'].to_numpy()

In [175]:
print(data.shape)
print(targets.shape)

(506, 13)
(506,)


In [176]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(data, targets, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In [177]:
inputs = torch.tensor(X_train)
targets = torch.tensor(Y_train).view(-1, 1)
test_inputs = torch.tensor(X_test)
test_targets =  torch.tensor(Y_test).view(-1, 1)

In [178]:
inputs.shape
#targets.shape

torch.Size([404, 13])

In [232]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_ds = TensorDataset(inputs, targets)

batch_size = 10
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [233]:
model = nn.Linear(inputs.shape[1], 1).double()
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0639,  0.2335, -0.0884,  0.1463, -0.1445,  0.2619,  0.0025, -0.0141,
          0.2510,  0.0653, -0.0508,  0.1817, -0.1827]], dtype=torch.float64,
       requires_grad=True)
Parameter containing:
tensor([-0.1517], dtype=torch.float64, requires_grad=True)


In [234]:
preds = model(inputs)
preds.shape

torch.Size([404, 1])

In [235]:
import torch.nn.functional as F

loss_fn = nn.MSELoss()
loss = loss_fn(model(inputs), targets)
print(loss)

opt = torch.optim.SGD(model.parameters(), lr=5e-7)

tensor(5313.8268, dtype=torch.float64, grad_fn=<MseLossBackward>)


In [236]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            
           
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [237]:
fit(300, model, loss_fn, opt, train_dl)

Epoch [10/300], Loss: 36.3694
Epoch [20/300], Loss: 97.5888
Epoch [30/300], Loss: 290.7516
Epoch [40/300], Loss: 31.6733
Epoch [50/300], Loss: 22.1384
Epoch [60/300], Loss: 27.0203
Epoch [70/300], Loss: 54.7964
Epoch [80/300], Loss: 7.4150
Epoch [90/300], Loss: 70.1011
Epoch [100/300], Loss: 58.8250
Epoch [110/300], Loss: 60.0403
Epoch [120/300], Loss: 125.9958
Epoch [130/300], Loss: 73.6696
Epoch [140/300], Loss: 175.0160
Epoch [150/300], Loss: 57.1772
Epoch [160/300], Loss: 31.5031
Epoch [170/300], Loss: 37.0605
Epoch [180/300], Loss: 33.6753
Epoch [190/300], Loss: 10.0122
Epoch [200/300], Loss: 33.5141
Epoch [210/300], Loss: 24.3242
Epoch [220/300], Loss: 7.8538
Epoch [230/300], Loss: 19.4230
Epoch [240/300], Loss: 18.7610
Epoch [250/300], Loss: 19.2992
Epoch [260/300], Loss: 36.3728
Epoch [270/300], Loss: 49.6830
Epoch [280/300], Loss: 46.8118
Epoch [290/300], Loss: 71.9305
Epoch [300/300], Loss: 9.3912


In [190]:
import torch

print(torch.__version__)

1.8.1+cu111


In [241]:
print(loss_fn(model(test_inputs), test_targets))

tensor(69.3814, dtype=torch.float64, grad_fn=<MseLossBackward>)
